In [1]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh

domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)

# Define the functional space
gdim = domain.topology.dim

from dolfinx import fem
from dolfinx.fem import Function, FunctionSpace, dirichletbc, locate_dofs_topological, form
import ufl
from ufl import grad, div, nabla_grad, dx, inner, dot

Vel = ufl.VectorElement("Lagrange", domain.ufl_cell(), 2)
Pa = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)
V, Q = FunctionSpace(domain, Vel), FunctionSpace(domain, Pa)

In [ ]:
import dolfinx

fdim = gdim - 1

inlet_marker  = 10 
walls_marker  = 20
#outlet_marker = 30

boundaries = [(inlet_marker,  lambda x: np.isclose(x[0], 0.)),
              (walls_marker,  lambda x: np.isclose(x[1], -R)),
              (walls_marker,  lambda x: np.isclose(x[1],  R)),
              (outlet_marker, lambda x: np.isclose(x[0], length))]

facet_indices, facet_markers = [], []
for (marker, locator) in boundaries:
    facets = dolfinx.mesh.locate_entities(domain, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full(len(facets), marker))
facet_indices = np.array(np.hstack(facet_indices), dtype=np.int32)
facet_markers = np.array(np.hstack(facet_markers), dtype=np.int32)
sorted_facets = np.argsort(facet_indices)
ft = dolfinx.mesh.meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])
